In [23]:
import tensorflow as tf
import numpy as np
tf.__version__

'1.6.0'

## first try

In [7]:
def get_data():
    x_data=np.float32(np.random.rand(2,100))
    y_data=np.dot([0.100,0.200],x_data)+0.300
    return x_data,y_data

x_data,y_data=get_data()

In [12]:
# 通过tensorflow搭建起vgg

b=tf.Variable(tf.zeros([1]))
W=tf.Variable(tf.random_uniform([1,2],-1.0,1.0))

y=tf.matmul(W,x_data)+b
# 定义loss

# 这里应该是为了计算batch的时候，算一个batch的均值
# 所以采用reduce-mean的吧

loss=tf.reduce_mean(tf.square(y-y_data))
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.5)
train = optimizer.minimize(loss)

In [20]:
# 初始化所有变量
# 这里不支持自定义初始化方式
init=tf.global_variables_initializer()

In [26]:

with tf.Session() as sess:
    sess.run(init)
    # 训练步骤
    for step in range(0,201):
        sess.run(train)
        if step%20 ==0:
            print(step,sess.run(W),sess.run(b))

0 [[0.5204204 0.5038343]] [-0.14161567]
20 [[0.22159123 0.3312029 ]] [0.1733526]
40 [[0.13583937 0.24765971]] [0.25813115]
60 [[0.11116112 0.21665157]] [0.2860473]
80 [[0.10359608 0.20570986]] [0.29533035]
100 [[0.10118156 0.20193957]] [0.29843363]
120 [[0.10039245 0.20065567]] [0.29947397]
140 [[0.1001311  0.20022109]] [0.29982322]
160 [[0.10004394 0.20007446]] [0.2999406]
180 [[0.10001474 0.20002505]] [0.29998]
200 [[0.10000497 0.20000844]] [0.29999328]


## Easy try 1:

    matrix multiply

In [28]:
# define matirx

matrix1=tf.constant([[1,2],[3,4]])
matrix2=tf.constant([[5,6],[7,8]])

# 默认是2个矩阵的乘法，所以直接乘就可以了
product=tf.matmul(matrix1,matrix2)

In [32]:
with tf.Session() as sess:
    # 所以其实梯度下降的时候，这里返回的是loss值吧？
    result=sess.run(product)
    print(result)


[[19 22]
 [43 50]]


## Easy try 2:

accumulation

In [39]:
'''
定义图里面 op的计算
'''

# 获得一个累加器
conter=tf.Variable(0,name='counter')
one=tf.constant(1)

'''
这里看起来比较麻烦可以认为是tf里面没有+=这个操作，所有对象的拷贝都是深拷贝
'''
# 累加
one_more=tf.add(conter,one)
# 赋值 
final_step=tf.assign(conter,one_more)

# 变量初始化
init=tf.global_variables_initializer()

In [38]:
'''
sess的执行逻辑是，只需要让她执行最后一步的op就行了。
它会根据op之间的依赖关系，一直回溯到最开始的constant或者variable，
来开始计算
'''
with tf.Session() as sess:
    sess.run(init)
    for _ in range(3):
        print(sess.run(final_step))

1
2
3


## Easy try 3:

multi fetch

In [42]:
'''
可以同时执行多个op
但是initial貌似必须显示的优先执行
'''
with tf.Session() as sess:
    sess.run(init)
    print(sess.run([product,final_step]))

[array([[19, 22],
       [43, 50]]), 1]


## Easy try 4：

feed-placeholder

In [58]:
# 根据shape来进行设定输入的tensor的shape，dtype来设置参数类型
input1=tf.placeholder(shape=(1,2),dtype=tf.float32)
input2=tf.placeholder(shape=(2,1),dtype=tf.float32)
output=tf.matmul(input1,input2)

In [55]:
test1=np.asarray([[1,2]])
test2=np.asarray([[2],[1]])

In [57]:
with tf.Session() as sess:
    result=sess.run(output,feed_dict={input1:test1,input2:test2})
    print(result)

[[4.]]
